In [1]:
import sqlite3
import pandas as pd
from pathlib import Path

DB_PATH = '../../chembl_36/chembl_36_sqlite/chembl_36.db'

print(f"  Size: {Path(DB_PATH).stat().st_size / (1024**3):.1f} GB")

conn = sqlite3.connect(DB_PATH)

  Size: 27.7 GB


In [2]:
import pandas as pd

In [3]:
query = """

SELECT DISTINCT
    md.molregno as drug_internal_id,
    md.chembl_id as drug_id,
    md.pref_name as drug_name,
    cs.canonical_smiles as smile
FROM molecule_dictionary md
JOIN compound_structures cs ON md.molregno = cs.molregno
WHERE md.max_phase = 4
  AND md.molecule_type = 'Small molecule'
  AND cs.canonical_smiles IS NOT NULL
ORDER BY md.chembl_id;

"""

approved_drugs = pd.read_sql(query, conn)
print(f"Found {len(approved_drugs)} approved small molecule drugs")
print(approved_drugs.head().to_string(index=False))

Found 3127 approved small molecule drugs
 drug_internal_id      drug_id             drug_name                                          smile
           111185   CHEMBL1000            CETIRIZINE    O=C(O)COCCN1CCN(C(c2ccccc2)c2ccc(Cl)cc2)CC1
           165474 CHEMBL100116           PENTAZOCINE             CC(C)=CCN1CCC2(C)c3cc(O)ccc3CC1C2C
           111482   CHEMBL1002        LEVOSALBUTAMOL              CC(C)(C)NC[C@H](O)c1ccc(O)c(CO)c1
           111491   CHEMBL1003 CLAVULANATE POTASSIUM O=C([O-])[C@H]1/C(=C/CO)O[C@@H]2CC(=O)N21.[K+]
           111498   CHEMBL1004            DOXYLAMINE                 CN(C)CCOC(C)(c1ccccc1)c1ccccn1


In [10]:
second_query = """ -- Drug-Protein interactions with binding strength (NO action_type)
SELECT 
    md.molregno as drug_internal_id,
    md.chembl_id as drug_id,
    md.pref_name as drug_name,
    
    td.tid as protein_internal_id,
    td.chembl_id as protein_id,
    td.pref_name as protein_name,
    
    -- Binding strength (aggregated from multiple measurements)
    MAX(act.pchembl_value) as pchembl_max,
    ROUND(AVG(act.pchembl_value), 2) as pchembl_avg,
    MIN(act.standard_value) as best_value,
    COUNT(DISTINCT act.activity_id) as num_measurements,
    
    -- Quality
    MAX(ass.confidence_score) as confidence

FROM activities act
JOIN assays ass ON act.assay_id = ass.assay_id
JOIN target_dictionary td ON ass.tid = td.tid
JOIN molecule_dictionary md ON act.molregno = md.molregno

WHERE md.max_phase = 4
  AND td.target_type = 'SINGLE PROTEIN'
  AND td.organism = 'Homo sapiens'
 AND act.pchembl_value >= 5.5
 AND ass.confidence_score >= 8
  
GROUP BY 
    md.molregno, md.chembl_id, md.pref_name,
    td.tid, td.chembl_id, td.pref_name

ORDER BY md.chembl_id, pchembl_max DESC;

"""

drugs_interactions = pd.read_sql(second_query, conn)
print(f"Found {len(drugs_interactions)} drug-protein interactions")
drugs_interactions

Found 11703 drug-protein interactions


,drug_internal_id,drug_id,drug_name,protein_internal_id,protein_id,protein_name,pchembl_max,pchembl_avg,best_value,num_measurements,confidence
0,111185,CHEMBL1000,CETIRIZINE,127,CHEMBL231,Histamine H1 receptor,8.23,7.50,5.89,12,9
1,111185,CHEMBL1000,CETIRIZINE,102,CHEMBL1941,Histamine H2 receptor,5.73,5.73,1851.40,1,8
2,111185,CHEMBL1000,CETIRIZINE,107,CHEMBL224,5-hydroxytryptamine receptor 2A,5.67,5.67,2117.80,1,8
3,165474,CHEMBL100116,PENTAZOCINE,137,CHEMBL237,Kappa-type opioid receptor,8.66,8.06,2.20,3,9
4,165474,CHEMBL100116,PENTAZOCINE,129,CHEMBL233,Mu-type opioid receptor,8.57,7.65,2.70,12,9
...,...,...,...,...,...,...,...,...,...,...,...
11698,110803,CHEMBL998,LORATADINE,11912,CHEMBL3401,Nuclear receptor subfamily 1 group I member 2,5.52,5.52,3000.00,1,9
11699,110803,CHEMBL998,LORATADINE,11180,CHEMBL2047,Bile acid receptor,5.51,5.51,3070.00,1,9
11700,164035,CHEMBL99946,LEVOMILNACIPRAN,100,CHEMBL222,Sodium-dependent noradrenaline transporter,7.98,7.59,10.50,3,9
11701,164035,CHEMBL99946,LEVOMILNACIPRAN,121,CHEMBL228,Sodium-dependent serotonin transporter,6.50,6.50,320.00,2,9


In [19]:
df = pd.read_csv("/home/joe/projects/pharmacology-graph/drug_nodes.csv")

df.head()

,drug_internal_id,drug_id,drug_name,smile
0,111185,CHEMBL1000,CETIRIZINE,O=C(O)COCCN1CCN(C(c2ccccc2)c2ccc(Cl)cc2)CC1
1,165474,CHEMBL100116,PENTAZOCINE,CC(C)=CCN1CCC2(C)c3cc(O)ccc3CC1C2C
2,111482,CHEMBL1002,LEVOSALBUTAMOL,CC(C)(C)NC[C@H](O)c1ccc(O)c(CO)c1
3,111491,CHEMBL1003,CLAVULANATE POTASSIUM,O=C([O-])[C@H]1/C(=C/CO)O[C@@H]2CC(=O)N21.[K+]
4,111498,CHEMBL1004,DOXYLAMINE,CN(C)CCOC(C)(c1ccccc1)c1ccccn1


# Smile Embedding 

In [20]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

# Load model + tokenizer once (outside the function for efficiency)
model_name = "seyonec/ChemBERTa-zinc-base-v1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.eval()

@torch.no_grad()
def get_smiles_embedding(smiles: str) -> np.ndarray:
    """
    Generate a molecular embedding for a given SMILES string using ChemBERTa.

    Args:
        smiles (str): SMILES representation of a molecule.
    
    Returns:
        np.ndarray: 1D embedding vector.
    """
    inputs = tokenizer(smiles, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    embedding = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
    return embedding


df["embedding"] = df["smile"].map(get_smiles_embedding)

print(df["embedding"][0].shape)
print(df.head())

/home/joe/projects/pharmacology-graph/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(768,)
   drug_internal_id       drug_id              drug_name  \
0            111185    CHEMBL1000             CETIRIZINE   
1            165474  CHEMBL100116            PENTAZOCINE   
2            111482    CHEMBL1002         LEVOSALBUTAMOL   
3            111491    CHEMBL1003  CLAVULANATE POTASSIUM   
4            111498    CHEMBL1004             DOXYLAMINE   

                                            smile  \
0     O=C(O)COCCN1CCN(C(c2ccccc2)c2ccc(Cl)cc2)CC1   
1              CC(C)=CCN1CCC2(C)c3cc(O)ccc3CC1C2C   
2               CC(C)(C)NC[C@H](O)c1ccc(O)c(CO)c1   
3  O=C([O-])[C@H]1/C(=C/CO)O[C@@H]2CC(=O)N21.[K+]   
4                  CN(C)CCOC(C)(c1ccccc1)c1ccccn1   

                                           embedding  
0  [1.0936161, 1.0498636, 0.45691708, -0.6111154,...  
1  [1.0808558, 0.38841933, 0.26941997, -0.3636055...  
2  [0.6716983, 0.57236403, 0.80348116, -1.1774188...  
3  [0.30589342, 0.44845665, -0.026959993, -0.4769...  
4  [1.1095629, 1.5414737, 0.4867429, -

In [21]:
# Convert embeddings to a 2D NumPy array
X = np.vstack(df["embedding"].values)
np.save("chemberta_embeddings.npy", X)

# Eval embedding

In [ ]:
# --- Embedding Quality Evaluation for SMILES (ChemBERTa) ---
import numpy as np
import pandas as pd
from tqdm import tqdm

# Baselines: RDKit Morgan + scaffold
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from rdkit.Chem.Scaffolds import MurckoScaffold

# Metrics + models
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from scipy.stats import spearmanr


# -----------------------------
# Helpers
# -----------------------------
def morgan_fp(smiles, radius=2, nBits=2048):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    return AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits)

def tanimoto(fp_a, fp_b):
    return DataStructs.TanimotoSimilarity(fp_a, fp_b)

def scaffold_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    scaf = MurckoScaffold.GetScaffoldForMol(mol)
    return Chem.MolToSmiles(scaf) if scaf is not None else None

def to_matrix_of_embeddings(series):
    # series of 1D arrays -> (N, d) matrix
    return np.vstack(series.values)

def cosine_similarity_matrix(X):
    Xn = X / (np.linalg.norm(X, axis=1, keepdims=True) + 1e-8)
    return Xn @ Xn.T

def recall_at_k(X, labels, k=10):
    Xn = X / (np.linalg.norm(X, axis=1, keepdims=True) + 1e-8)
    sims = Xn @ Xn.T
    np.fill_diagonal(sims, -np.inf)
    idx_topk = np.argpartition(-sims, k, axis=1)[:, :k]
    any_hit = []
    mean_frac = []
    labels = np.asarray(labels)
    for i in range(len(labels)):
        nbrs = idx_topk[i]
        same = (labels[nbrs] == labels[i])
        any_hit.append(np.any(same))
        mean_frac.append(np.mean(same))
    return float(np.mean(any_hit)), float(np.mean(mean_frac))

def pick_first_existing_label_column(df, candidates=("ATC","atc","label","target","indication")):
    for c in candidates:
        if c in df.columns:
            return c
    return None


# -----------------------------
# 0) Basic checks and prep
# -----------------------------
assert "smile" in df.columns, "df must contain a 'smile' column."
assert "embedding" in df.columns, "df must contain an 'embedding' column of 1D vectors."

# Filter to valid rows (non-null SMILES and embeddings)
valid_mask = df["smile"].notna() & df["embedding"].apply(lambda x: isinstance(x, np.ndarray) and x.ndim == 1)
dfv = df.loc[valid_mask].reset_index(drop=True)

if len(dfv) < 10:
    raise ValueError("Need at least ~10 valid molecules for meaningful metrics.")

X = to_matrix_of_embeddings(dfv["embedding"])   # (N, d)
smiles_list = dfv["smile"].tolist()

print(f"[Info] Using {len(dfv)} molecules with embeddings for evaluation.")


# -----------------------------
# 1) RDKit Morgan baseline + agreement
# -----------------------------
print("\n[1] Morgan baseline + agreement with embedding space")
fps = [morgan_fp(s) for s in tqdm(smiles_list, desc="Computing Morgan FPs")]
valid_fp_idx = [i for i,f in enumerate(fps) if f is not None]

if len(valid_fp_idx) < 10:
    print(" - Skipping agreement: too few valid Morgan fingerprints.")
else:
    # Subsample for pairwise distance correlation (to keep O(n^2) small)
    sub_idx = np.random.choice(valid_fp_idx, size=min(2000, len(valid_fp_idx)), replace=False)
    X_sub = X[sub_idx]
    fps_sub = [fps[i] for i in sub_idx]

    # Cosine distance in embedding space
    sims = cosine_similarity_matrix(X_sub)
    cos_dist = 1.0 - sims
    # Tanimoto distance in Morgan space
    n = len(sub_idx)
    tan_dist = np.zeros((n, n), dtype=np.float32)
    for i in range(n):
        for j in range(i+1, n):
            sim = tanimoto(fps_sub[i], fps_sub[j])
            d = 1.0 - sim
            tan_dist[i, j] = tan_dist[j, i] = d

    # Spearman on upper triangle
    iu = np.triu_indices(n, 1)
    rho, pval = spearmanr(cos_dist[iu], tan_dist[iu])
    print(f" - Spearman(cosine, 1–Tanimoto) = {rho:.3f} (p={pval:.1e})   "
          f"[0.3–0.7 is common; very high => mirrors structure only]")


# -----------------------------
# 2) Scaffolds + clustering quality
# -----------------------------
print("\n[2] Scaffold structure & clustering")
scaffolds = [scaffold_smiles(s) for s in smiles_list]
dfv["scaffold"] = scaffolds
scaf_valid = dfv["scaffold"].notna()
scaf_labels = dfv.loc[scaf_valid, "scaffold"].values
X_scaf = X[scaf_valid.values]

if len(np.unique(scaf_labels)) <= 1 or len(scaf_labels) < 20:
    print(" - Skipping clustering: need >= 20 molecules and >1 unique scaffold.")
else:
    k = len(np.unique(scaf_labels))
    k = min(k, max(2, int(np.sqrt(len(scaf_labels)))))  # cap clusters for stability
    km = KMeans(n_clusters=k, n_init="auto", random_state=0).fit(X_scaf)
    sil = silhouette_score(X_scaf, km.labels_, metric="cosine")
    ari = adjusted_rand_score(scaf_labels, km.labels_)
    nmi = normalized_mutual_info_score(scaf_labels, km.labels_)
    print(f" - Silhouette (cosine): {sil:.3f}")
    print(f" - ARI vs scaffold:     {ari:.3f}")
    print(f" - NMI vs scaffold:     {nmi:.3f}")


# -----------------------------
# 3) Label-based evaluation (if label column exists)
# -----------------------------
label_col = pick_first_existing_label_column(dfv)
print("\n[3] Label-based metrics" + (f" using '{label_col}'" if label_col else " [no label column found]"))

if label_col:
    y = dfv[label_col].astype(str).values
    # Require at least 2 classes and enough samples
    if len(np.unique(y)) >= 2 and len(y) >= 50:
        # kNN (cosine)
        try:
            knn = KNeighborsClassifier(n_neighbors=10, metric="cosine")
            knn.fit(X, y)
            knn_acc = knn.score(X, y)  # optimistic (no split); useful as locality sanity check
            print(f" - kNN(10) self-accuracy: {knn_acc:.3f}  [optimistic, no split]")
        except Exception as e:
            print(f" - kNN skipped: {e}")

        # Linear probe (CV). NOTE: This is NOT scaffold split; for real tasks, build a scaffold split index.
        try:
            skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
            scores = []
            for tr, te in skf.split(X, y):
                scaler = StandardScaler(with_mean=True, with_std=True)
                Xtr = scaler.fit_transform(X[tr]); Xte = scaler.transform(X[te])
                clf = LogisticRegression(max_iter=5000)
                clf.fit(Xtr, y[tr])
                scores.append(clf.score(Xte, y[te]))
            print(f" - Linear probe (5x CV) acc: {np.mean(scores):.3f} ± {np.std(scores):.3f}")
        except Exception as e:
            print(f" - Linear probe skipped: {e}")

        # Retrieval: Recall@10 for same label
        try:
            r_any, r_mean = recall_at_k(X, y, k=10)
            print(f" - Retrieval Recall@10(any) = {r_any:.3f} | mean label fraction@10 = {r_mean:.3f}")
        except Exception as e:
            print(f" - Retrieval skipped: {e}")
    else:
        print(" - Skipping: not enough label diversity/samples.")
else:
    print(" - Provide a label column (e.g., ATC/indication/target) to enable kNN/linear-probe/retrieval.")


# -----------------------------
# 4) Quick summary guidance
# -----------------------------
print("\n[Guide]")
print(" - If Spearman(cosine, 1–Tanimoto) is moderate (≈0.3–0.7), you’re not just duplicating fingerprints.")
print(" - Higher Silhouette / ARI / NMI vs scaffolds indicates clear structural organization.")
print(" - kNN / Linear probe beating Morgan (under proper scaffold split) = strong pharmacology signal.")
print(" - Retrieval Recall@10 high for same label = neighbors align with mechanism/ATC.")


[Info] Using 3127 molecules with embeddings for evaluation.

[1] Morgan baseline + agreement with embedding space


Computing Morgan FPs:   0%|          | 0/3127 [00:00<?, ?it/s][05:36:06] DEPRECATION WARNING: please use MorganGenerator
[05:36:06] DEPRECATION WARNING: please use MorganGenerator
[05:36:06] DEPRECATION WARNING: please use MorganGenerator
[05:36:06] DEPRECATION WARNING: please use MorganGenerator
[05:36:06] DEPRECATION WARNING: please use MorganGenerator
[05:36:06] DEPRECATION WARNING: please use MorganGenerator
[05:36:06] DEPRECATION WARNING: please use MorganGenerator
[05:36:06] DEPRECATION WARNING: please use MorganGenerator
[05:36:06] DEPRECATION WARNING: please use MorganGenerator
[05:36:06] DEPRECATION WARNING: please use MorganGenerator
[05:36:06] DEPRECATION WARNING: please use MorganGenerator
[05:36:06] DEPRECATION WARNING: please use MorganGenerator
[05:36:06] DEPRECATION WARNING: please use MorganGenerator
[05:36:06] DEPRECATION WARNING: please use MorganGenerator
[05:36:06] DEPRECATION WARNING: please use MorganGenerator
[05:36:06] DEPRECATION WARNING: please use MorganGene

 - Spearman(cosine, 1–Tanimoto) = 0.494 (p=0.0e+00)   [0.3–0.7 is common; very high => mirrors structure only]

[2] Scaffold structure & clustering


[05:36:09] WARNING: not removing hydrogen atom without neighbors
[05:36:09] WARNING: not removing hydrogen atom without neighbors
[05:36:09] WARNING: not removing hydrogen atom without neighbors
[05:36:09] WARNING: not removing hydrogen atom without neighbors


 - Silhouette (cosine): 0.111
 - ARI vs scaffold:     0.063
 - NMI vs scaffold:     0.593

[3] Label-based metrics [no label column found]
 - Provide a label column (e.g., ATC/indication/target) to enable kNN/linear-probe/retrieval.

[Guide]
 - If Spearman(cosine, 1–Tanimoto) is moderate (≈0.3–0.7), you’re not just duplicating fingerprints.
 - Higher Silhouette / ARI / NMI vs scaffolds indicates clear structural organization.
 - kNN / Linear probe beating Morgan (under proper scaffold split) = strong pharmacology signal.
 - Retrieval Recall@10 high for same label = neighbors align with mechanism/ATC.


# protein Embedding

In [26]:
from transformers import AutoTokenizer, AutoModel
import torch

model_checkpoint = "facebook/esm2_t12_35M_UR50D"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModel.from_pretrained(model_checkpoint)

sequence = "MGDVEKGKKIFIMKCSQCHTVEKGGKHKTGPNE"  # example protein sequence

inputs = tokenizer(sequence, return_tensors="pt", add_special_tokens=True)
with torch.no_grad():
    outputs = model(**inputs)
    embedding = outputs.last_hidden_state.mean(dim=1)  # average pooling

print(embedding)  # -> torch.Size([1, 480]) depending on model size

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t12_35M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor([[-1.0509e-01, -3.2571e-02,  2.1700e-01,  2.8530e-02, -3.6089e-02,
         -1.8468e-01, -1.4655e-01,  1.0215e-02,  8.4399e-02, -9.0626e-02,
          8.8253e-04,  2.4982e-02,  1.3211e-02, -1.2763e-01,  2.0745e-01,
         -1.9708e-01,  1.4129e-02, -6.4314e-02,  5.7334e-02,  9.8735e-02,
         -1.4660e-01, -8.6152e-02, -1.8656e-02, -8.0563e-02, -6.7036e-02,
         -7.6510e-02,  1.3121e-02, -6.4535e-02, -1.7816e-01, -2.4527e-01,
         -1.0789e-01, -5.9125e-02, -5.0317e-02,  7.6339e-03, -2.2403e-01,
         -7.5385e-02, -1.2701e-02, -3.8951e-02, -1.5408e-02, -9.5410e-02,
          4.0891e-03,  1.0999e-01, -1.8453e-01,  1.1015e-01,  7.5123e-02,
          7.7984e-02,  4.9880e+00, -1.7739e-02,  2.4734e-02,  1.3436e-01,
          1.1330e-01,  9.4437e-02, -8.3311e-02,  9.8582e-02,  7.7189e-02,
          4.4058e-02, -8.7686e-02, -2.6493e-02, -1.5234e-01,  2.2334e-02,
         -6.7814e-02, -1.0475e-01, -2.7717e-02, -3.2317e-01,  9.1917e-02,
          1.0289e-01,  1.0820e-02, -1.